INSTALL
- FAISS - 
- Sent2Vec

In [1]:
import numpy as np
import faiss   

In [2]:
import os
import time
name = "Faiss-EM-ECB-en-fr"+ str(time.strftime("%Y%m%d-%H%M"))

!mkdir $name
path = os.getcwd() + "/" + name
os.chdir(path)

!pwd
os.chdir(path)

/home/chinh/clc_data/Script/Faiss-EM-ECB-en-fr20210305-0918


In [3]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/ecb-en-fr.txt.zip
!mkdir data_bin
!unzip 'ecb-en-fr.txt.zip' -d 'data_bin'

--2021-03-05 09:18:30--  https://github.com/hoangtrungchinh/clc_data/raw/master/ecb-en-fr.txt.zip
Resolving github.com (github.com)... 52.74.223.119
Connecting to github.com (github.com)|52.74.223.119|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/ecb-en-fr.txt.zip [following]
--2021-03-05 09:18:31--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/ecb-en-fr.txt.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18865629 (18M) [application/zip]
Saving to: ‘ecb-en-fr.txt.zip’

ecb-en-fr.txt.zip   100%[===================>]  17,99M  5,44MB/s    in 3,5s    

2021-03-05 09:18:34 (5,20 MB/s) - ‘ecb-en-fr.txt.zip’ saved [18865629/18865629]

Archive:

In [4]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
fr_files = sorted(glob.glob("data_bin/*.fr"))
print(en_files)
print(fr_files)

lst_en = []
lst_fr = []

start = time.time()

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line.strip())

for file in fr_files:
  with open(file) as file_in:
    for line in file_in:
      lst_fr.append(line.strip())

print(len(lst_en))
print(len(lst_fr))

['data_bin/ECB.en-fr.en']
['data_bin/ECB.en-fr.fr']
195960
195960


In [5]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, fr_train, fr_test_valid =  train_test_split(lst_en, lst_fr, test_size=0.2, random_state=123)
en_valid, en_test, fr_valid, fr_test = train_test_split(en_test_valid, fr_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(fr_train))
print(len(en_valid))
print(len(fr_valid))
print(len(en_test))
print(len(fr_test))
print(en_train[20])
print(fr_train[20])
print(en_valid[-1])
print(fr_valid[-1])
print(en_test[-1])
print(fr_test[-1])

156768
156768
19596
19596
19596
19596
ECB / 2006/5
BCE / 2006/5
When the service-providing NCB receives a positive or negative acknowledgement from the receiving NCB / ECB it shall forward such an acknowledgement to the connected NCB . Transfer of responsibility for the execution of a payment order For cross-border payments initiated by a participant in the connected NCB 's RTGS system , responsibility for the execution of a payment order shall pass from the connected NCB to the service-providing NCB at the moment the connected NCB 's account with the service-providing NCB is debited , and thereafter it shall pass to the receiving NCB / ECB in accordance
Transfert de responsabilité pour l' exécution d' un ordre de paiement Pour les paiements transfrontaliers émis par un participant au système RBTR de la BCN connectée , la responsabilité de l' exécution d' un ordre de paiement est transférée de la BCN connectée à la BCN prestataire de services , au moment où le compte de la BCN connecté

In [6]:
file_variable = [en_train,fr_train,en_valid,fr_valid,en_test,fr_test]
file_names = ["en_train","fr_train","en_valid","fr_valid","en_test","fr_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item)
      f.write("\n")
    
!ls -al

total 87192
drwxrwxr-x 3 chinh chinh     4096 Thg 3   5 09:20 .
drwxrwxr-x 5 chinh chinh     4096 Thg 3   5 09:19 ..
drwxrwxr-x 2 chinh chinh     4096 Thg 3   5 09:18 data_bin
-rw-rw-r-- 1 chinh chinh 18865629 Thg 3   5 09:18 ecb-en-fr.txt.zip
-rw-rw-r-- 1 chinh chinh  3262752 Thg 3   5 09:20 en_test
-rw-rw-r-- 1 chinh chinh 25944504 Thg 3   5 09:20 en_train
-rw-rw-r-- 1 chinh chinh  3208009 Thg 3   5 09:20 en_valid
-rw-rw-r-- 1 chinh chinh  3798492 Thg 3   5 09:20 fr_test
-rw-rw-r-- 1 chinh chinh 30424593 Thg 3   5 09:20 fr_train
-rw-rw-r-- 1 chinh chinh  3754500 Thg 3   5 09:20 fr_valid


In [7]:
start = time.time()
import sent2vec
model = sent2vec.Sent2vecModel()
model.load_model('../../../wiki_unigrams.bin')

print('=== Ending, Total time (second): ', time.time() - start)

=== Ending, Total time (second):  2.237156867980957


In [8]:
start = time.time()
srcfile = "en_train"
tarfile = "fr_train"
lst_srcfile = en_train
lst_tarfile = fr_train

vec_lst_srcfile = model.embed_sentences(lst_srcfile)
print('=== Ending, Total time (second): ', time.time() - start)

=== Ending, Total time (second):  1.0949366092681885


Test Cosin similarity

In [9]:
import faiss

start = time.time()
index = faiss.IndexFlatL2(vec_lst_srcfile.shape[1])
index.ntotal
faiss.normalize_L2(vec_lst_srcfile)
index.add(vec_lst_srcfile)
k = 6
distance, index = index.search(vec_lst_srcfile, k)

print('=== Ending, Total time (second): ', time.time() - start)
print(len(distance))
print(len(index))

print(index)
print(distance)

=== Ending, Total time (second):  84.76998233795166
156768
156768
[[     0  80931  88493 109915  99819  38424]
 [     1  97476 130294 121189    896  79800]
 [     2  52203  61963 152892 124051  60662]
 ...
 [156765 131521 149402 124053  74169  87306]
 [156766   1200   2081 132448  44413   4274]
 [ 25656  99540 156767 112645  85733  50680]]
[[2.3841858e-07 1.1861241e-01 2.0717204e-01 4.9055958e-01 5.1163995e-01
  6.5848005e-01]
 [1.1920929e-07 3.4690642e-01 3.6197495e-01 3.6693537e-01 3.9221680e-01
  4.0737975e-01]
 [0.0000000e+00 3.5041332e-01 3.5041332e-01 4.7909641e-01 4.7909641e-01
  4.8441112e-01]
 ...
 [1.1920929e-07 2.4547136e-01 2.4547136e-01 2.6380062e-01 2.6419234e-01
  2.6419234e-01]
 [0.0000000e+00 5.1288569e-01 5.1459229e-01 5.4631281e-01 6.3336837e-01
  6.3634980e-01]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 1.1095762e-01 7.9572654e-01
  8.2322824e-01]]


In [10]:
j = k-1
# TEST
for i in range(10):
    a = vec_lst_srcfile[i]
    b = vec_lst_srcfile[index[i][j]]
    dist_squared = np.sum(np.square(a - b))
    print("index:",index[i][j], "distance:",distance[i][j], "tested:",dist_squared)

index: 38424 distance: 0.65848005 tested: 0.65848017
index: 79800 distance: 0.40737975 tested: 0.4073795
index: 60662 distance: 0.48441112 tested: 0.4844109
index: 121734 distance: 0.6218971 tested: 0.621897
index: 47430 distance: 0.26732183 tested: 0.26732177
index: 98424 distance: 0.15686142 tested: 0.1568611
index: 145902 distance: 0.7232028 tested: 0.72320306
index: 21958 distance: 0.9152571 tested: 0.91525733
index: 30758 distance: 0.07613635 tested: 0.07613627
index: 77002 distance: 0.17340136 tested: 0.17340136


In [11]:
start = time.time()
saperate = " || "

src_label = "S"
emb_label = "E"
thres_arr = [0.9, 0.8, 0.7, 0.6]
for i in range(len(thres_arr)):
    threshold = thres_arr[i]
    out_file_write = open("en_train_EM_"+str(thres_arr[i]), 'w')
    out_file_write_score = open("en_train_EM_score_"+str(thres_arr[i]), 'w')
    out_file_write_factor = open("en_train_EM_factor_"+str(thres_arr[i]), 'w')
    num_gth_thres = 0

    for i in range(len(lst_srcfile)):
        score = distance[i][1]
        content = ""
        content_label_arr = []
        if score >= threshold:
            num_gth_thres += 1
            best_simi_index = index[i][1]
            content = lst_srcfile[i] + saperate + lst_tarfile[best_simi_index]

            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label.append(emb_label)
            content_label = content_label + [emb_label] * len(lst_tarfile[best_simi_index].split())

            content_label_arr = " ".join(content_label)
        else:
            content = lst_srcfile[i]
            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label_arr = " ".join(content_label)

        out_file_write.writelines(content+ "\n")    
        out_file_write_factor.writelines(content_label_arr + "\n")
        out_file_write_score.writelines(str(score) + "\n")

    print('threshold =', threshold, "EM sentences =", num_gth_thres, "/", len(lst_srcfile), num_gth_thres/len(lst_srcfile), "%" )

    out_file_write.close()
    out_file_write_score.close()
    out_file_write_factor.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)
    


threshold = 0.9 EM sentences = 3973 / 156768 0.02534318228209839 %
threshold = 0.8 EM sentences = 11661 / 156768 0.0743838028169014 %
threshold = 0.7 EM sentences = 23288 / 156768 0.14855072463768115 %
threshold = 0.6 EM sentences = 36308 / 156768 0.23160338844662176 %
=== Ending, Total time (second):  13.76828384399414
